In this experiment, our agent doesn't inially know whether the top or bottom lamp is lit. This knowledge isn't available to the state representation. It has to devise a plan that works regardless of whether it's in a Top world or a Bottom world. Note that this will be impossible unless it creates an internal memory register that tracks which lamp it's seen.

In [1]:
from LampButtonBoxState import LampButtonBoxState
from StateSuperposition import StateSuperposition

In [2]:
boxstate1 = LampButtonBoxState()
boxstate1._internal_switch = 0

boxstate2 = LampButtonBoxState()
boxstate2._internal_switch = 1

boxsuper_0 = StateSuperposition()
boxsuper_0.add_state(boxstate1)
boxsuper_0.add_state(boxstate2)

boxsuper_0.observe()

[(["LID_OPEN"], Super:2)]

In [3]:
boxsuper_1 = boxsuper_0.act("CLOSE_LID")
boxsuper_1.observe()

[(["LID_CLOSED", "LIGHT_TOP"], Super:1),
 (["LID_CLOSED", "LIGHT_BOTTOM"], Super:1)]

In [4]:
boxsuper_2 = boxsuper_1.act("OPEN_LID")
boxsuper_2.observe()

[(["LID_OPEN"], Super:2)]

Ah, but here's the rub! At this point, the search diverges. We explore the superpositions resulting from the *observations*, not those resulting from the action!

In [5]:
boxsuper_3 = boxsuper_2.act("PRESS_BUTTON_BOTTOM")
boxsuper_3.observe()

[(["LID_OPEN", "ZAP"], Super:1), (["LID_OPEN", "TREAT"], Super:1)]

In [6]:
a = [3,21,1]
a.sort()
a

[1, 3, 21]

In this game, it's a guaranteed win. We take no chances.

When performing search, the rules are:
* An observation is victorious if it contains a TREAT.
* An observation is victorious if its child superposition is victorious.
* An observation is defeated if it contains a ZAP.
* An observation is defeated if its child superposition is defeated.
* A state is victorious if its observation is victorious.
* A state is defeated if its observation is defeated.
* A state is victorious if there exists at least one action performed from that state that is victorious.
* A state is defeated if every action from that state is defeated.
* An action is victorious if every state it leads to is victorious.
* An action is defeated if any state it leads to is defeated.
* A superposition is victorious if all of its comprised states are victorious.
* A superposition is defeated if any of its comprised states are defeated.
* A superposition is victorious if all of its observations are victorious. It means that, no matter what we will observe, we will have a route to victory from it.
* A superposition is defeated if any of its observations are defeated. It means that there's a possibility that, if we enter this superposition, the world might throw a situation at us that we won't be able to recover from.

And remember, kids:
* We expand superstates from *observations*, not based on actions!
* We are ultimately looking for the next *action* to take. All other considerations are secondary.

In [98]:
# Takes a state that was arrived at through some action. ("Start" is an implicit action).
class ObservationSearchNode:
    def __init__(self):
        self.statesuperposition = None
        
        self.is_victory = False
        self.is_defeat = False
        
        self.parent_action_node = None
        self.action_in_parent = None
        
        # Tuples of observation object and action search nodes.
        self.children_of_observations = []
        
    
    # Determines whether or not this node contains a termination condition.
    # Returns True if this termination condition was newly detected and needs
    # to be propagated to the parent (or was propagated), otherwise False.
    def evaluate_termination(self, recurse=True):
        print("Evaluating termination")
        if self.is_victory or self.is_defeat:
            # No change.
            return False
        
        if len(self.children_of_observations) == 0:
            return False        
        
        all_victory = True
        for observation, childnode in self.children_of_observations:
            print(observation)
            print(f"Does observation have zap? {observation.has('ZAP')}")
            if observation.has('ZAP') or childnode.is_defeat:
                print("This node is defeated")
                # If ANY observation is defeated, this search node is defeated.
                self.is_defeat = True
                if recurse and self.parent_action_node:
                    self.parent_action_node.evaluate_termination(True)
                # Return True to indicate we've changed.
                return True
            
            if not (observation.has('TREAT') or childnode.is_victory):
                all_victory = False
                # We can't break or return because we might still be defeated.
        
        if all_victory:
            # If ALL observations are victorious, this search node is victorious.
            self.is_victory = True
            if recurse and self.parent_action_node:
                self.parent_action_node.evaluate_termination(True)            
            # Return True to indicate we've changed.
            return True
        
        # No change.
        return False

    
    # Perform an observation in this superposition, and produce new
    # superpositions that correspond to states that are grouped by the
    # common observations that they would produce.
    def collapse_superposition(self):
        newnodes = []
        if self.statesuperposition is None:
            return newnodes
        
        obssets = self.statesuperposition.observe()
        for observation, nextsuperposition in obssets:
            actionnode = ActionSearchNode()
            actionnode.statesuperposition = nextsuperposition
            actionnode.parent_observation_node = self
            actionnode.observation_in_parent = observation
            self.children_of_observations.append( (observation, actionnode) )
            newnodes.append(actionnode)

        # Now that we have children, evaluate our terminations.
        self.evaluate_termination(True)
        return newnodes
            
            

# Takes a superposition that resulted from an observation,
# and performs an action in it.
class ActionSearchNode:
    def __init__(self):
        self.statesuperposition = None
        
        self.is_victory = False
        self.is_defeat = False
        
        self.parent_observation_node = None
        self.observation_in_parent = None
        
        self.children_of_actions = {}
        
    # Determines whether or not this node contains a termination condition.
    # Returns True if this termination condition was newly detected and needs
    # to be propagated to the parent (or was propagated), otherwise False.    
    def evaluate_termination(self, recurse=True):
        if self.is_victory or self.is_defeat:
            # No change.
            return False
        
        if len(self.children_of_actions) == 0:
            return False
        
        all_defeat = True
        for childnode in self.children_of_actions.values():
            if childnode.is_victory:
                # If any action child is victorious, then this node is victorious.
                self.is_victory = True
                if recurse and self.parent_observation_node:
                    self.parent_observation_node.evaluate_termination(True)
                # Return True to indicate we've changed.
                return True
            
            if not childnode.is_defeat:
                all_defeat = False

        if all_defeat:
            # If all action children are defeated, then this node is defeated.
            self.is_defeat = True
            if recurse and self.parent_observation_node:
                self.parent_observation_node.evaluate_termination(True)
            # Return True to indicate we've changed.
            return True
        
        # No change.
        return False
        
    # Go through possible actions from possible states, and act them
    # on this superposition.
    def brainstorm_actions(self):
        newnodes = []
        if self.statesuperposition is None:
            return newnodes
        
        actionoptions = self.statesuperposition.generate_possible_actions()
        for action in actionoptions:
            nextsuperposition = self.statesuperposition.act(action)
            observenode = ObservationSearchNode()
            observenode.statesuperposition = nextsuperposition
            observenode.parent_action_node = self
            observenode.action_in_parent = action
            self.children_of_actions[action] = observenode
            newnodes.append(observenode)
            
        # Now that we have children, evaluate our terminations.
        self.evaluate_termination(True)
        return newnodes
            
        
        
        
        
    
        

In [99]:
node1 = ObservationSearchNode()
node1.statesuperposition = boxsuper_0
node1.evaluate_termination()

Evaluating termination


False

In [100]:
node1.collapse_superposition()

Evaluating termination
["LID_OPEN"]
Does observation have zap? False


In [101]:
(node1.is_victory, node1.is_defeat)

(False, False)

In [102]:
node1.children_of_observations[0][1].brainstorm_actions()

In [103]:
(node1.children_of_observations[0][1].children_of_actions['PRESS_BUTTON_TOP'].is_victory,
 node1.children_of_observations[0][1].children_of_actions['PRESS_BUTTON_TOP'].is_defeat)

(False, False)

In [104]:
node1.children_of_observations[0][1].children_of_actions['PRESS_BUTTON_TOP'].collapse_superposition()

Evaluating termination
["LID_OPEN", "TREAT"]
Does observation have zap? False
["LID_OPEN", "ZAP"]
Does observation have zap? True
This node is defeated


In [105]:
node1.children_of_observations[0][1].children_of_actions['PRESS_BUTTON_TOP'].is_defeat

True

In [106]:
node1.is_defeat

False

In [107]:
node1.children_of_observations[0][1].children_of_actions

{'PRESS_BUTTON_TOP': <__main__.ObservationSearchNode at 0x2455d07ff10>,
 'CLOSE_LID': <__main__.ObservationSearchNode at 0x2455c582df0>,
 'PRESS_BUTTON_BOTTOM': <__main__.ObservationSearchNode at 0x2455c582e80>,
 'OPEN_LID': <__main__.ObservationSearchNode at 0x2455d0eef10>}

In [108]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].collapse_superposition()

Evaluating termination
["LID_CLOSED", "LIGHT_TOP"]
Does observation have zap? False
["LID_CLOSED", "LIGHT_BOTTOM"]
Does observation have zap? False


In [109]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations

[(["LID_CLOSED", "LIGHT_TOP"], <__main__.ActionSearchNode at 0x2455a5f58b0>),
 (["LID_CLOSED", "LIGHT_BOTTOM"],
  <__main__.ActionSearchNode at 0x2455c509310>)]

In [110]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[0][1].brainstorm_actions()

In [111]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[0][1].children_of_actions["OPEN_LID"].collapse_superposition()

Evaluating termination
["LID_OPEN"]
Does observation have zap? False


In [112]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[0][1].children_of_actions["OPEN_LID"].children_of_observations[0][1].brainstorm_actions()

In [113]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[0][1].children_of_actions["OPEN_LID"].children_of_observations[0][1].children_of_actions["PRESS_BUTTON_TOP"].collapse_superposition()

Evaluating termination
["LID_OPEN", "TREAT"]
Does observation have zap? False
Evaluating termination
["LID_OPEN"]
Does observation have zap? False
Evaluating termination
["LID_CLOSED", "LIGHT_TOP"]
Does observation have zap? False
["LID_CLOSED", "LIGHT_BOTTOM"]
Does observation have zap? False


In [114]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[0][1].children_of_actions["OPEN_LID"].children_of_observations[0][1].children_of_actions["PRESS_BUTTON_TOP"].is_victory

True

In [115]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[0][1].children_of_actions["OPEN_LID"].is_victory

True

In [117]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].is_victory

False

In [118]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[1][1].brainstorm_actions()
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[1][1].children_of_actions["OPEN_LID"].collapse_superposition()
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[1][1].children_of_actions["OPEN_LID"].children_of_observations[0][1].brainstorm_actions()
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[1][1].children_of_actions["OPEN_LID"].children_of_observations[0][1].children_of_actions["PRESS_BUTTON_TOP"].collapse_superposition()

Evaluating termination
["LID_OPEN"]
Does observation have zap? False
Evaluating termination
["LID_OPEN", "ZAP"]
Does observation have zap? True
This node is defeated


In [119]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[1][1].children_of_actions["OPEN_LID"].children_of_observations[0][1].is_defeat

False

In [120]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[1][1].children_of_actions["OPEN_LID"].children_of_observations[0][1].children_of_actions["PRESS_BUTTON_BOTTOM"].collapse_superposition()

Evaluating termination
["LID_OPEN", "TREAT"]
Does observation have zap? False
Evaluating termination
["LID_OPEN"]
Does observation have zap? False
Evaluating termination
["LID_CLOSED", "LIGHT_TOP"]
Does observation have zap? False
["LID_CLOSED", "LIGHT_BOTTOM"]
Does observation have zap? False
Evaluating termination
["LID_OPEN"]
Does observation have zap? False


In [121]:
node1.children_of_observations[0][1].children_of_actions["CLOSE_LID"].children_of_observations[1][1].children_of_actions["OPEN_LID"].children_of_observations[0][1].children_of_actions["PRESS_BUTTON_BOTTOM"].is_victory

True

In [122]:
node1.is_victory

True

In [132]:
[(k,v.is_victory) for k,v in node1.children_of_observations[0][1].children_of_actions.items()]

[('PRESS_BUTTON_TOP', False),
 ('CLOSE_LID', True),
 ('PRESS_BUTTON_BOTTOM', False),
 ('OPEN_LID', False)]